In [1]:
import pickle

import pandas as pd

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import root_mean_squared_error

In [2]:
from sklearn.pipeline import make_pipeline

In [3]:
import mlflow

mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("green-taxi-duration")

<Experiment: artifact_location='s3://mlflow-artifacts-remote4/1', creation_time=1761250616806, experiment_id='1', last_update_time=1761250616806, lifecycle_stage='active', name='green-taxi-duration', tags={}>

In [4]:
def read_dataframe(year: int, month: int) -> pd.DataFrame:
    url = f'https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_{year}-{month:02d}.parquet'
    df = pd.read_parquet(url)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    return df


def prepare_dictionaries(df: pd.DataFrame):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts

In [5]:
df_train = read_dataframe(2021, 1)
df_val = read_dataframe(2021, 2)

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

dict_train = prepare_dictionaries(df_train)
dict_val = prepare_dictionaries(df_val)

In [7]:
with mlflow.start_run():
    params = dict(max_depth=20, n_estimators=100, min_samples_leaf=10, random_state=0)
    mlflow.log_params(params)

    pipeline = make_pipeline(
        DictVectorizer(),
        RandomForestRegressor(**params, n_jobs=-1)
    )

    pipeline.fit(dict_train, y_train)
    y_pred = pipeline.predict(dict_val)

    rmse = root_mean_squared_error(y_pred, y_val)
    print(params, rmse)
    mlflow.log_metric('rmse', rmse)

    mlflow.sklearn.log_model(pipeline, name="model")

{'max_depth': 20, 'n_estimators': 100, 'min_samples_leaf': 10, 'random_state': 0} 6.7558229919200725


2025/10/23 20:57:25 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run fun-wolf-468 at: http://127.0.0.1:5000/#/experiments/1/runs/e341a1a864e64fb8bb73d57bc5917e60
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


In [8]:
from mlflow.tracking import MlflowClient


In [10]:
MLFLOW_TRACKING_URI = 'http://127.0.0.1:5000'
RUN_ID = 'e341a1a864e64fb8bb73d57bc5917e60'

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [11]:
path = client.download_artifacts(run_id=RUN_ID, path='dict_vectorizer.bin')

MlflowException: Failed to download artifacts from path 'dict_vectorizer.bin', please ensure that the path is correct.

In [22]:
with open(path, 'rb') as f_out:
    dv = pickle.load(f_out)

In [23]:
dv

DictVectorizer()